In [16]:
%%capture
import tensorflow as tf
import keras
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import pickle
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import classification_report

import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt

## Load Saved Models

In [3]:
model_paths = ["rnn_w_GRU.keras",
               "rnn_w_LSTM.keras",
               "rnn_w_GRU_and_LSTM.keras",
               "cnn.keras"]
models = [keras.models.load_model(path) for path in model_paths]

## Dataset

In [4]:
df = pd.read_csv('balanced_dataset.csv')
df.head()

,class,tweet
0,0,queer gaywad
1,0,alsarabsss hes beaner smh tell hes mexican
2,0,youre fucking gay blacklisted hoe holding tehg...
3,0,lmfaoooo hate black people theres black people...
4,0,least im nigger lmfao


## Split data

In [8]:
features = df['tweet']
target = df['class']

X_train, X_val, Y_train, Y_val = train_test_split(features,
												target,
												test_size=0.2,
												random_state=22)
X_train.shape, X_val.shape

# make labels categorical
Y_train = to_categorical(Y_train, num_classes=3)
Y_val = to_categorical(Y_val, num_classes=3)

## Tokenization

In [10]:
with open('tokenizer.pkl', 'rb') as handle:
    token = pickle.load(handle)

In [12]:
# generating token embeddings
max_words = 5000
max_len = 100

X_train_seq = token.texts_to_sequences(X_train)
X_val_seq = token.texts_to_sequences(X_val)

X_train_padded = pad_sequences(X_train_seq, maxlen=max_len)
X_val_padded = pad_sequences(X_val_seq, maxlen=max_len)

## Evaluate Models

In [22]:
results = {}
for i, model in enumerate(models):
    pred = model.predict(X_val_padded)
    pred_class = np.argmax(pred, axis=1)
    Y_val_class = np.argmax(Y_val, axis=1)
    # Assuming you have test_labels in a compatible format
    report = classification_report(Y_val_class, pred_class, output_dict=True)
    results[model_paths[i]] = report

75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 30ms/step
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
75/75 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step
75/75 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step


In [23]:
comparison_df = pd.DataFrame({
    model_name: [results[model_name]['weighted avg']['precision'],
                 results[model_name]['weighted avg']['recall'],
                 results[model_name]['weighted avg']['f1-score']]
    for model_name in results
}, index=["Precision","Recall","F1-Score"])

print(comparison_df)

           rnn_w_GRU.keras  rnn_w_LSTM.keras  rnn_w_GRU_and_LSTM.keras  \
Precision         0.890575          0.887554                  0.883935   
Recall            0.889586          0.886658                  0.882058   
F1-Score          0.887988          0.885016                  0.880056   

           cnn.keras  
Precision   0.897141  
Recall      0.896278  
F1-Score    0.894653  
